In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from keras import backend as K
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, AveragePooling1D,Lambda
from tensorflow.keras import Model

In [ ]:

(X_train, y_train), (X_test, y_test) = tensorflow.keras.datasets.mnist.load_data()
X_train = X_train.reshape(X_train.shape[0], 28,28,1)
X_test = X_test.reshape(X_test.shape[0],28,28,1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
x_train =X_train/255
x_test =X_test/255

In [ ]:
#creating triplets 
#create (positive,anchor,negative)
#create list of different labels
import random

def create_triplets(x_train,y_train,train=True):
    pos_train=[]
    neg_train=[]
    anch_train=[]
    df = pd.DataFrame(columns=['pos','anch','neg'])
    triplets=[]
    #lets 50 training example for 4 
    if train == True:
        sample=5000
    else:
        sample=400
    train_pair=[]
    labels=[0,1,2,3,4,5,6,7,8,9]
    for label in labels:
        
        p=[i for i,x in enumerate(y_train) if x==label] 
        p_50=random.sample(p,sample)
        p_50_img=list(x_train[p_50])
        [pos_train.append(t) for t in p_50_img]
        
        a_50=random.sample(p,sample)
        a_50_img=list(x_train[a_50])
        [anch_train.append(t) for t in a_50_img]
        
        n=[i for i,x in enumerate(y_train) if x!=label]
        n_50=random.sample(n,sample)
        n_50_img=list(x_train[n_50])
        [neg_train.append(t) for t in n_50_img]
    return pos_train,anch_train,neg_train
        #df.append({'pos':p_50_img,'anch': a_50_img,'neg': n_50_img}, ignore_index=True)

pos_train,anch_train,neg_train=create_triplets(x_train,y_train)
pos_test,anch_test,neg_test=create_triplets(x_test,y_test,train=False)
         


In [ ]:
class CustomDataGen(tensorflow.keras.utils.Sequence):
    def __init__(self,pos_train,anch_train,neg_train,batch_size):
        self.pos=pos_train
        self.anch=anch_train
        self.neg=neg_train
        self.batch_size=batch_size
        self.n = len(self.pos)

        
    def __getitem__(self, index):
        
        pos = self.pos[index * self.batch_size:(index + 1) * self.batch_size]
        anch=self.anch[index * self.batch_size:(index + 1) * self.batch_size]
        neg=self.neg[index * self.batch_size:(index + 1) * self.batch_size]
        label = np.ones(batch_size)
        return [np.asarray(pos,dtype='float32'),np.asarray(anch,dtype='float32'),np.asarray(neg,dtype='float32')],label
    def __len__(self):
        return np.int(self.n // self.batch_size)
traingen=CustomDataGen(pos_train=pos_train,anch_train=anch_train,neg_train=neg_train,batch_size=8)
evalgen =CustomDataGen(pos_train=pos_test,anch_train=anch_test,neg_train=neg_test,batch_size=8)       
        

In [ ]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D,ZeroPadding2D
from tensorflow.keras.layers import Input, Lambda

#from keras.datasets import mnist
from tensorflow.keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [ ]:
def triplet_loss(x, alpha = 0.2):
    positive,anchor,negative = x
    # distance between the anchor and the positive
    dp = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    dn = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = dp-dn+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss
def identity_loss(y_true, y_pred):
    return K.mean(y_pred)


In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='basemodel_plot.png', show_shapes=True, show_layer_names=True)
plot_model(model2, to_file='tripletmodel_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
embedding_dim = 2 
LR = 0.0005
batch_size=8
alpha = 0.2 
input_shape=(28, 28,1)
def embedding_model():
  # Simple convolutional model 
  # used for the embedding model.
  input_shape=(28,28,1)
  model = Sequential()
  model.add(Convolution2D(32, (3, 3), activation='relu',
                        input_shape=input_shape))
  model.add(Convolution2D(32, (3, 3), activation='relu'))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(12))
  return model


model=embedding_model()

P = Input(shape=input_shape, name = 'anchorPositive')
enc_P = model(P)
A = Input(shape=input_shape, name = 'anchor')
enc_A = model(A)
N = Input(shape=input_shape, name = 'anchorNegative')
enc_N = model(N)
loss = Lambda(triplet_loss)([enc_P,enc_A,enc_N]) 
model2 = Model(inputs=[P,A,N], outputs=loss)
model2.compile(loss=identity_loss, optimizer=Adam(LR))
history = model2.fit_generator(traingen,validation_data=evalgen,epochs=200, 
                    verbose=1,steps_per_epoch=20,validation_steps=30)

In [ ]:
model2.summary()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Validation Losses',size = 20)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')
plt.show()

In [ ]:
model.save('triplet_loss.h5')

In [ ]:
sample_size = 500

X_test = model.predict(x_test[:sample_size].reshape(-1,28,28,1))
y_test=y_test[:sample_size]

In [ ]:
from sklearn.manifold import TSNE
tsne=TSNE()
test_tsne_embeds = tsne.fit_transform(X_test)

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import pandas as pd


def scatter(x, labels, subtitle=None):
    # Create a scatter plot of all the 
    # the embeddings of the model.
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))
    # We create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0,alpha = 0.5, s=40,
                    c=palette[labels.astype(np.int)] )
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')
    
    

scatter(test_tsne_embeds, y_test)